# **Sampling methods: particle filter**

## Nearly-constant velocity Example : Ground Truth

In [1]:
import numpy as np 
from datetime import datetime 
from datetime import timedelta

start_time = datetime.now().replace(microsecond=0)

In [2]:
np.random.seed(1313)

Now let's initialise Stone Soup ground-truth and transition models 

In [3]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

transition_model = CombinedLinearGaussianTransitionModel(
    [ConstantVelocity(0.05), ConstantVelocity(0.05)]
)

timesteps = [start_time]
truth = GroundTruthPath(
    [GroundTruthState([0,1,0,1], timestamp=start_time)]
)

Create the truth path 

In [4]:
for k in range (1, 21): 
    timesteps.append(start_time+timedelta(microseconds=k))
    truth.append(
        GroundTruthState(
            transition_model.function(truth[k-1], noise=True, time_interval=timedelta(seconds=1)), 
            timestamp=timesteps[k]
        )
    )

Plot the ground truth

In [5]:
from stonesoup.plotter import AnimatedPlotterly
plotter = AnimatedPlotterly(timesteps, tail_length=0.3)
plotter.plot_ground_truths(truth, [0, 2])
plotter.fig

Now we initialize the bearing, range sensor using the appropiate measurement model. 

In [7]:
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.detection import Detection

sensor_x = 50
sensor_y = 0

measurement_model = CartesianToBearingRange(
    ndim_state=4,
    mapping=(0, 2),
    noise_covar=np.diag([np.radians(0.2), 1]),
    translation_offset=np.array([[sensor_x], [sensor_y]])
)

Now we can populate the measurement array

In [8]:
measurements = []
for state in truth:
    measurement = measurement_model.function(state, noise=True)
    measurements.append(Detection(measurement, timestamp=state.timestamp,
                                  measurement_model=measurement_model))

Plot the measurements

In [9]:
plotter.plot_measurements(measurements, [0, 2])
plotter.fig

## Set up Particle Filter

We create: 
* ParticlePredictor
* ParticleUpdater

These require a TransitionModel and MeasurementModel as before. To cope with sample sparsity we include a resampler -> SystematicResampler

## Use of Effective Sample Size resampler (ESS)

Resampling removes particles with low weight and duplicare the particles with high weight. A side effect is that the additional variance is addes. Use of `SystematicResampler`at each time-step means that the additional variance is being introduced when it may not necessarily be required. To reduce the additional variance, it may be optimal to resamle less frequently.

The Effective Sample Size resampler `ESSResampler`compares the variance of the unnormalised weights of the particles to a pre-specified threshold, and only resamples when the variance is greater than this threshold. This threshold is often calculated by the ESS criterion (at time n) given by: 

$$
ESS = (\sum_{i=1}^{N}(W_{n}^{i})²)^{-1}
$$

In [10]:
from stonesoup.predictor.particle import ParticlePredictor
predictor = ParticlePredictor(transition_model)

from stonesoup.resampler.particle import ESSResampler
resampler = ESSResampler()

from stonesoup.updater.particle import ParticleUpdater
updater = ParticleUpdater(measurement_model, resampler)


## Initialise a prior
We have to create a prior estimate. This is a `ParticleState` which describes the state as a distribution of particles using `StateVectors` and weights. This is sampled from the Gaussian distribution (using the same parameters where we had in the previous examples).

In [11]:
from scipy.stats import multivariate_normal

from stonesoup.types.numeric import Probability  # Similar to a float type
from stonesoup.types.state import ParticleState
from stonesoup.types.array import StateVectors

number_particles = 1000

# Sample from the prior Gaussian distribution
samples = multivariate_normal.rvs(np.array([0, 1, 0, 1]),
                                  np.diag([1.5, 0.5, 1.5, 0.5]),
                                  size=number_particles)

# Create prior particle state.
prior = ParticleState(state_vector=StateVectors(samples.T),
                      weight=np.array([Probability(1/number_particles)]*number_particles),
                      timestamp=start_time)

## Run the tracker
Run the predict and update steps, propagating the collection of particles and resampling when told to (at every step). 

In [12]:
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.track import Track

track = Track()
for measurement in measurements:
    prediction = predictor.predict(prior, timestamp=measurement.timestamp)
    hypothesis = SingleHypothesis(prediction, measurement)
    post = updater.update(hypothesis)
    track.append(post)
    prior = track[-1]

Plot result with the sample points at each iteration. Can also change the 'plot history' to True if wanted. 

In [13]:
plotter.plot_tracks(track, [0, 2], particle=True, plot_history=False)
plotter.fig